# Runnable Primitives
# RunnableSequence

In [1]:
!pip install -q langchain langchain-community google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [3]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [4]:
import os
from google.colab import userdata

# Access the API key from Colab's Secrets Manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

In [11]:
from IPython.display import display
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

prompt1 = PromptTemplate(
    template="write a joke about {topic}",
    input_variables=["topic"]
)

parser = StrOutputParser()

# RunnableSequence(prompt1 | model | parser) works too

chain = RunnableSequence(
    prompt1 , model , parser
)

result = chain.invoke({"topic":"science"})
display(result)

"Why don't scientists trust atoms?\n\nBecause they make up everything!"

In [9]:
!pip install grandalf
chain.get_graph().print_ascii()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.2 MB/s eta 0:00:00
      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [ ]:
from IPython.display import display
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

prompt1 = PromptTemplate(
    template="write a joke about {topic}",
    input_variables=["topic"]
)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template="explain the following joke-{text}",
    input_variables=["text"]
)

chain = RunnableSequence(
    prompt1, model, parser, prompt2, model, parser
)

result = chain.invoke({"topic":"AI"})
display(result)

# RunnableParallel
already done in chains

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence


prompt1 = PromptTemplate(
    template="Give one line tweet about {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="generate one line linkedin post about {topic}",
    input_variables=["topic"]
)

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)


parser = StrOutputParser()


parallel_chain = RunnableParallel({
    "tweet": RunnableSequence(
        prompt1, model, parser
    ),
    "linkedin":RunnableSequence(
        prompt2, model, parser
    )
    })

result = parallel_chain.invoke({"topic":"AI"})

display(result)

# RunnablePassthrough
give same output as input

In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence, RunnablePassthrough

passthrough = RunnablePassthrough()

print(passthrough.invoke(2))

2


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence, RunnablePassthrough

prompt1 = PromptTemplate(
    template="write a joke about {topic}",
    input_variables=["topic"]
)

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template="explain the following joke-{text}",
    input_variables=["text"]
)


jokes_generator_chain = RunnableSequence(
    prompt1, model, parser
)

parallel_chain = RunnableParallel({
    "joke": RunnablePassthrough(),
    "explanation":RunnableSequence(
        prompt2, model , parser
    )
})


final_chain = RunnableSequence(jokes_generator_chain, parallel_chain)

final_chain.invoke({"topic":"cricket"})

# RunnableLambda: convert python function to runnable functions
eg. uses to preprocess the data : add custom logic to the runnable

In [12]:
from langchain.schema.runnable import RunnableLambda

def words_counter(text):
  return len(text.split())



runnable_word_counter = RunnableLambda(words_counter)

print(runnable_word_counter.invoke("this is awesome"))

3


In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence, RunnablePassthrough, RunnableLambda

prompt = PromptTemplate(
    template="write a joke about {topic}",
    input_variables=["topic"]
)

def words_counter(text):
  return len(text.split())

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.5)

parser = StrOutputParser()

joke_generator_chain = RunnableSequence(prompt, model , parser)

parallel_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    # "word_count":RunnableLambda(words_counter)
    "word_count":RunnableLambda(lambda x: len(x.split()))
})



final_chain = RunnableSequence(joke_generator_chain, parallel_chain)

print(final_chain.invoke({"topic":"AI"}))

{'joke': "Here are a few options, pick your favorite!\n\n**Option 1 (Classic AI trope):**\n> Why did the AI break up with its calculator?\n> Because it said they just didn't *compute* their feelings.\n\n**Option 2 (Current AI issue):**\n> My AI told me a joke the other day.\n> It was hilarious, but then I fact-checked it and found out the punchline was completely made up.\n\n**Option 3 (Relatable job market humor):**\n> My boss replaced me with an AI.\n> Now I just sit around all day, prompting it to write my resume.\n\n**Option 4 (Literal interpretation):**\n> Why did the AI cross the road?\n> To optimize its route efficiency, but it still couldn't tell you *why* the chicken did.", 'word_count': 122}


you are getting more jokes because of the term temperature = 0.5 so decrease it to 0 or remove it

# RunnableBranch
if else statement of langchain

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableSequence, RunnablePassthrough, RunnableLambda, RunnableBranch

prompt1 = PromptTemplate(
    template="write a simple report on {topic}",
    input_variables=["topic"]
)


prompt2 = PromptTemplate(
    template="summarize the following {text}",
    input_variables=["text"]
)


model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()


report_gen_chain = RunnableSequence(prompt1 , model , parser)

branch_chain = RunnableBranch(
    (lambda x: len(x.split()) > 500, RunnableSequence(prompt2, model, parser)),
    (RunnablePassthrough())
)


final_chain = RunnableSequence(report_gen_chain | branch_chain)

final_chain.invoke({"topic":"Influence of AI"})

"This report outlines the profound and pervasive influence of Artificial Intelligence (AI) across various sectors.\n\n**Key areas of AI's influence include:**\n\n*   **Work and Economy:** Driving automation for efficiency, transforming job roles by displacing some while creating new ones, and fostering economic growth through innovation.\n*   **Daily Life:** Powering smart devices, personalizing recommendations, and enhancing navigation and future transportation.\n*   **Healthcare:** Assisting in faster diagnosis, drug discovery, personalized treatments, and improving operational efficiency in medical facilities.\n*   **Creativity and Education:** Generating diverse content (text, images, music) and enabling personalized learning experiences for students.\n*   **Research and Development:** Accelerating scientific discovery through vast data analysis and complex system simulations.\n\n**However, the report also highlights significant challenges and considerations:**\n\n*   **Job Displac

In [17]:
final_chain.get_graph().print_ascii()

      +-------------+      
      | PromptInput |      
      +-------------+      
             *             
             *             
             *             
    +----------------+     
    | PromptTemplate |     
    +----------------+     
             *             
             *             
             *             
+------------------------+ 
| ChatGoogleGenerativeAI | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | StrOutputParser |    
    +-----------------+    
             *             
             *             
             *             
        +--------+         
        | Branch |         
        +--------+         
             *             
             *             
             *             
     +--------------+      
     | BranchOutput |      
     +--------------+      


# LCEL
RunnableSequence is most common runnable so replace

RunnableSequence(r1 , r2, r3) ------> r1 | r2 | r3 use this syntax instead
pipe operator